![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [18]:
# Run the cells below first

In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [20]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [21]:
# Start coding here
# Use as many cells as you need

In [22]:
# Define CNN model
class GarmentClassifier(nn.Module):
    def __init__(self):
        super(GarmentClassifier, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # (28x28) → (28x28)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # → (14x14)
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # → (14x14)
            nn.ReLU(),
            nn.MaxPool2d(2, 2)   # → (7x7)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)  # 10 classes
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x


In [23]:

# Data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Instantiate model, loss, optimizer
model = GarmentClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train for 1 epoch to reduce run time
model.train()
for epoch in range(1):  # 1 epoch
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


In [24]:

# Evaluation and predictions
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy().tolist())  # ✅ Cast to list
        true_labels.extend(labels.cpu().numpy().tolist())

# Now predictions is a true Python list


In [25]:

# Compute accuracy, per-class precision, and recall
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average=None, zero_division=0).tolist()
recall = recall_score(true_labels, predictions, average=None, zero_division=0).tolist()

# Show metrics
accuracy, precision, recall


(0.8814,
 [0.8127984718242598,
  0.9897959183673469,
  0.8645235361653272,
  0.9151450053705693,
  0.7968596663395485,
  0.9827760891590679,
  0.622822299651568,
  0.9392752203721841,
  0.974,
  0.9618473895582329],
 [0.851, 0.97, 0.753, 0.852, 0.812, 0.97, 0.715, 0.959, 0.974, 0.958])

In [27]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [28]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Define CNN
class GarmentClassifier(nn.Module):
    def __init__(self):
        super(GarmentClassifier, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# Load data
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Train
model = GarmentClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(1):  # Keep epochs low for fast runtime
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluate
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy().tolist())  # Ensuring it's a list
        true_labels.extend(labels.cpu().numpy().tolist())

# Metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average=None, zero_division=0).tolist()
recall = recall_score(true_labels, predictions, average=None, zero_division=0).tolist()

# Output
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision per class: {precision}")
print(f"Recall per class: {recall}")


Accuracy: 0.8790
Precision per class: [0.8451327433628318, 0.9927083333333333, 0.8558951965065502, 0.771004942339374, 0.7834862385321101, 0.9739478957915831, 0.7197740112994351, 0.9468405215646941, 0.954368932038835, 0.9572564612326043]
Recall per class: [0.764, 0.953, 0.784, 0.936, 0.854, 0.972, 0.637, 0.944, 0.983, 0.963]
